In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import random

In [2]:
# Define the root directories for training and testing
train_dir = 'C:/Users/Patrik/Desktop/Files/fei/Ing/2nd class/HNS/project/Dataset_Znacky/Train-adj-train-test/train'
test_dir = 'C:/Users/Patrik/Desktop/Files/fei/Ing/2nd class/HNS/project/Dataset_Znacky/Train-adj-train-test/test'

# Create a list of unique class labels for training data
unique_labels_train = sorted(os.listdir(train_dir))

# Create a mapping from class labels to sequential integers for training data
label_to_int_train = {label: i for i, label in enumerate(unique_labels_train)}

# Create a list of unique class labels for test data
unique_labels_test = sorted(os.listdir(test_dir))

# Create a mapping from class labels to sequential integers for test data
label_to_int_test = {label: i for i, label in enumerate(unique_labels_test)}

# Define transforms for data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, data_dir, label_to_int, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []

        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)

            if os.path.isdir(class_dir):
                images = os.listdir(class_dir)
                random.shuffle(images)

                for img in images:
                    image_path = os.path.join(class_dir, img)
                    label = label_to_int[class_name]
                    self.samples.append((image_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label

# Create custom datasets for training and testing
train_dataset = CustomDataset(train_dir, label_to_int_train, data_transforms)
test_dataset = CustomDataset(test_dir, label_to_int_test, data_transforms)

# Create data loaders for training and testing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load a pretrained GoogLeNet model
model = models.googlenet(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(unique_labels_train))  # Output units match the number of unique labels for training



In [ ]:
# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
device = torch.device("cpu")  # ("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")



In [33]:
# Test the model on the testing data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 95.21%
